# 02 - Preprocessing

### 1. Clean Data and Correct Dtypes
Some columns that are not useful or have the wrong data type, we must correct this.

### 2. Imputing and Duplicates
Since missing percentage in each feature (column) in our dataset is low, imputation is a better option to process missing values.

For imputation, given that our variables are mostly categorical we have two strategies
- Replacing all missing values with `Unknown` (We are going to use this one for the base)
- Replacing all missing values with the `most frequent` value

### 3. Encoding
For encoding, we are doing
- Frequency encoding for features with "High-Cardinality"
- Onehot encoding for binominal and nominal features
- Ordinal encoding for `FAMI_ESTRATOFAMILIA` since it describes a status class

### Data Loading and Exploration

In [15]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy
%matplotlib inline

from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from typing import Union

sns.set_palette('dark')

In [3]:
# Load data
df = pd.read_csv('data/train.csv')
df.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,medio-bajo


In [4]:
df.shape

(692500, 17)

In [5]:
df.dtypes

ID                                 int64
PERIODO                            int64
ESTU_PRGM_ACADEMICO               object
ESTU_PRGM_DEPARTAMENTO            object
ESTU_VALORMATRICULAUNIVERSIDAD    object
ESTU_HORASSEMANATRABAJA           object
FAMI_ESTRATOVIVIENDA              object
FAMI_TIENEINTERNET                object
FAMI_EDUCACIONPADRE               object
FAMI_TIENELAVADORA                object
FAMI_TIENEAUTOMOVIL               object
ESTU_PRIVADO_LIBERTAD             object
ESTU_PAGOMATRICULAPROPIO          object
FAMI_TIENECOMPUTADOR              object
FAMI_TIENEINTERNET.1              object
FAMI_EDUCACIONMADRE               object
RENDIMIENTO_GLOBAL                object
dtype: object

In [6]:
# Look the number of missing values
df.isnull().sum()

ID                                    0
PERIODO                               0
ESTU_PRGM_ACADEMICO                   0
ESTU_PRGM_DEPARTAMENTO                0
ESTU_VALORMATRICULAUNIVERSIDAD     6287
ESTU_HORASSEMANATRABAJA           30857
FAMI_ESTRATOVIVIENDA              32137
FAMI_TIENEINTERNET                26629
FAMI_EDUCACIONPADRE               23178
FAMI_TIENELAVADORA                39773
FAMI_TIENEAUTOMOVIL               43623
ESTU_PRIVADO_LIBERTAD                 0
ESTU_PAGOMATRICULAPROPIO           6498
FAMI_TIENECOMPUTADOR              38103
FAMI_TIENEINTERNET.1              26629
FAMI_EDUCACIONMADRE               23664
RENDIMIENTO_GLOBAL                    0
dtype: int64

In [7]:
df.duplicated(subset=['FAMI_TIENEINTERNET', 'FAMI_TIENEINTERNET.1']).sum() / df.shape[0]

0.9999956678700361

### Preprocessing Utility Functions

In [8]:
import unicodedata
import re

def handle_tuition_price(text: str) -> str:
    special_cases = ['Menos de 500 mil', 'Más de 7 millones']
    matches = None
    pattern1 = r'(\d+\.?\d*)' # For special cases
    pattern2 = r'(\d+(?:\.\d+)?).* de (\d+(?:\.\d+)?)' # For other cases

    if text in special_cases:
        matches = re.search(pattern1, text)
    else:
        matches = re.search(pattern2, text)

    if matches:
        # Map values to classes
        # Bajo: menos de 2.5
        # Medio: mas de 2.5 y menos de 5.5
        # Alto: 5.5 en adelante
        groups = matches.groups()
        if '500' in groups or '1' in groups:
            text = 'Bajo'
            
        elif '4' in groups:
            text = 'Medio'
            
        elif '7' in groups:
            text = 'Alto'
            
    elif text == 'No pagó matrícula':
        text = 'Gratis'

    return text

def handle_work_hours(text: str) -> str:
    """Handles how is the work load of the student"""
    if text in ['0', 'Menos de 10 horas']:
        text = 'Baja'
    elif text == 'Entre 11 y 20 horas':
        text = 'Media'
    elif text in ['Más de 30 horas', 'Entre 21 y 30 horas']:
        text = 'Alta'
    
    return text

def norm_text(text: str) -> str:
    return unicodedata.normalize("NFKD", text).encode("ASCII", "ignore").decode("utf-8")

def handle_rare_values(X, col, threshold=0.1):
    """Assigns a value to rare values on a column"""
    
    percentages = X[col].value_counts(normalize=True) * 100
    group_fn = lambda x: 'OTRO' if percentages[x] < threshold else x

    return X[col].apply(group_fn)

def handle_parent_education(text: str) -> str:
    """Handles and assigns the correct education of a person"""
    # TODO: Check if completa and handle incompleta to get the value of the last
    # category.
    # Superior: Prosgrado, pregrado, tecnica, profesional
    # Media: Bachillerato
    # Basica: Primaria
    # Inicial: Si no termina primaria
    # No aplica: Si no tiene ninguna
    pattern = r'.* (completa|incompleta)'
    matches = re.search(pattern, text)

    superior_education = ['Postgrado', 'profesional', 'Tecnica']

    if matches:
        if 'completa' == matches.group(1):
            # Check if at least one level is in text
            if any(level in text for level in superior_education):
                text = 'Superior'
            elif 'Secundaria' in text:
                text = 'Media'
            elif 'Primaria' in text:
                text = 'Basica'
        elif 'incompleta' == matches.group(1):
            if any(level in text for level in superior_education):
                text = 'Media'
            elif 'Secundaria' in text:
                text = 'Basica'
            elif 'Primaria' in text:
                text = 'Ninguna'

    if text == 'Postgrado':
        text = 'Superior'

    if text in ['Unknown', 'No Aplica']:
        text = 'No sabe'

    if text == 'Ninguno':
        text = 'Ninguna'

    return text

In [9]:
def _check_X(X: Union[pd.DataFrame, np.generic, np.ndarray]) -> pd.DataFrame:
    """Checks what type of data structure is being pass to encoders."""
    # TODO: raise if not an accepted data structure.
    if isinstance(X, pd.DataFrame):
        X = X.copy()
    
    elif isinstance(X, (np.ndarray, np.generic)):
        # TODO: Check the shape for 0 - 1 dims.
        X = pd.DataFrame(X)

    return X


class FrequencyEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, normalize=False):
        self.freq_map = {}
        self.normalize = normalize

    def fit(self, X, y=None):
        # NOTE: Here we process data and fill encoder params.
        X = _check_X(X)
        # Select categorical variable columns from X
        selected_columns = X.select_dtypes(include=["object", "category"]).columns.to_list()
        for col in selected_columns:

            if not self.normalize:
                self.freq_map[col] = X[col].value_counts().to_dict()
            else:
                self.freq_map[col] = X[col].value_counts(normalize=True).to_dict()
        
        return self

    def transform(self, X, y=None):
        # NOTE: Here we return the data transformed with all the values
        # processed in the params.
        X = _check_X(X)
        # Replace original values by mapped values.
        for col, freq_map_values in self.freq_map.items():
            X[col] = X[col].map(freq_map_values)
        
        return X

### Preprocessing Process

In [10]:
def preprocess_features(X: pd.DataFrame) -> pd.DataFrame:
    """Cleans and preprocess training features"""
    # Create a dataframe copy
    X = X.copy()

    # Clean data and correct dtypes
    cols_to_drop = ['FAMI_TIENEINTERNET.1', 'ID']
    X = X.drop(cols_to_drop, axis=1)
    X['PERIODO'] = X['PERIODO'].astype(str).apply(lambda text: text[:4])
        

    # Imputation
    for col in ['ESTU_VALORMATRICULAUNIVERSIDAD', 'FAMI_ESTRATOVIVIENDA', 'ESTU_HORASSEMANATRABAJA',
               'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE']:
        X[col] = X[col].fillna('Unknown')
        if col == 'FAMI_ESTRATOVIVIENDA':
            X[col] = X[col].fillna('Sin Estrato')

    for col in ['FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL',
               'ESTU_PRIVADO_LIBERTAD', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR']:
        col_mode = X[col].mode()[0]
        X[col] = X[col].fillna(col_mode)

    # Cleaning and Transforming data
    for col in ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE']:
        X[col] = X[col].apply(norm_text) # Normalize text
        if col == 'ESTU_PRGM_DEPARTAMENTO':
            X[col] = handle_rare_values(X, col, threshold=1.5)
        elif col == 'ESTU_PRGM_ACADEMICO':
            X[col] = handle_rare_values(X, col)
    
    X['ESTU_VALORMATRICULAUNIVERSIDAD'] = X['ESTU_VALORMATRICULAUNIVERSIDAD'].apply(handle_tuition_price)
    X['ESTU_HORASSEMANATRABAJA'] = X['ESTU_HORASSEMANATRABAJA'].apply(handle_work_hours)
    X['FAMI_EDUCACIONMADRE'] = X['FAMI_EDUCACIONMADRE'].apply(handle_parent_education)
    X['FAMI_EDUCACIONPADRE'] = X['FAMI_EDUCACIONPADRE'].apply(handle_parent_education)

    # Binary encoding
    for col in ['FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL',
               'ESTU_PRIVADO_LIBERTAD', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR']:
        if col == 'ESTU_PRIVADO_LIBERTAD':
            X[col] = X[col].map({'S': 1, 'N': 0})
        else:
            X[col] = X[col].map({'Si': 1, 'No': 0})
    # Ordinal encoding
    encoder = OrdinalEncoder()
    ord_cols = ['FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCACIONMADRE', 'FAMI_EDUCACIONPADRE',
                'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA']
    
    X_ord = pd.DataFrame(encoder.fit_transform(X[ord_cols]), columns=ord_cols)
    X = pd.concat([X.drop(columns=ord_cols, axis=1), X_ord], axis=1)
    # Frequency encoding
    encoder = FrequencyEncoder(normalize=True)
    high_card_cols = ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO']
    X_high_card = encoder.fit_transform(X[high_card_cols])
    X = pd.concat([X.drop(columns=high_card_cols, axis=1), X_high_card], axis=1)
    # One-hot encoding
    X = pd.get_dummies(X, columns=['PERIODO'], dtype=int)
        
    return X

In [11]:
X = df.drop('RENDIMIENTO_GLOBAL', axis=1)
y = df.RENDIMIENTO_GLOBAL

In [12]:
X = preprocess_features(X)

In [13]:
X.sample(10)

,FAMI_TIENEINTERNET,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_ESTRATOVIVIENDA,FAMI_EDUCACIONMADRE,FAMI_EDUCACIONPADRE,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,PERIODO_2018,PERIODO_2019,PERIODO_2020,PERIODO_2021
151496,0,0,0,0,1,1,2.0,0.0,2.0,1.0,0.0,0.074868,0.019428,0,1,0,0
457676,1,1,1,0,0,1,5.0,4.0,4.0,3.0,0.0,0.016562,0.120732,0,1,0,0
348102,0,0,1,0,1,1,2.0,0.0,4.0,1.0,0.0,0.137743,0.017600,0,1,0,0
449979,1,1,0,0,0,1,3.0,4.0,4.0,3.0,1.0,0.002170,0.041629,0,1,0,0
527496,1,1,0,0,0,1,2.0,1.0,0.0,0.0,0.0,0.071204,0.407450,0,0,1,0
470044,1,1,1,0,0,1,2.0,1.0,1.0,3.0,1.0,0.005071,0.407450,0,0,0,1
216638,1,1,1,0,0,1,1.0,1.0,1.0,1.0,1.0,0.018758,0.041629,0,0,1,0
9407,1,1,1,0,1,1,2.0,4.0,2.0,1.0,2.0,0.110728,0.020286,0,0,0,1
195757,1,1,1,0,0,1,4.0,4.0,4.0,0.0,2.0,0.001460,0.407450,0,1,0,0
145359,1,1,1,0,1,1,2.0,1.0,1.0,3.0,0.0,0.030243,0.041629,0,1,0,0


## Base Model using Preprocessing

In [14]:
from pycaret.classification import setup, compare_models

In [20]:
label_encoder = LabelEncoder()

y_encoded = pd.DataFrame(label_encoder.fit_transform(y), columns=['RENDIMIENTO_GLOBAL'])

In [29]:
data = pd.concat([X, y_encoded], axis=1)
data.head(10)

,FAMI_TIENEINTERNET,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_ESTRATOVIVIENDA,FAMI_EDUCACIONMADRE,FAMI_EDUCACIONPADRE,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,PERIODO_2018,PERIODO_2019,PERIODO_2020,PERIODO_2021,RENDIMIENTO_GLOBAL
0,1,1,1,0,0,1,2.0,4.0,1.0,0.0,1.0,0.017597,0.407450,0,0,0,1,2
1,0,1,0,0,0,1,2.0,1.0,4.0,3.0,1.0,0.076887,0.059235,0,0,0,1,1
2,1,1,0,0,0,0,2.0,1.0,1.0,3.0,0.0,0.002465,0.407450,0,0,1,0,1
3,1,1,0,0,0,1,3.0,1.0,3.0,3.0,1.0,0.110728,0.041629,0,1,0,0,0
4,1,1,1,0,0,1,2.0,0.0,0.0,3.0,0.0,0.071204,0.120732,0,0,0,1,3
5,1,1,1,0,0,1,4.0,1.0,4.0,0.0,1.0,0.003674,0.120732,0,0,1,0,2
6,1,1,1,0,1,1,1.0,4.0,1.0,3.0,0.0,0.010500,0.097707,1,0,0,0,0
7,1,1,0,0,1,0,1.0,0.0,2.0,1.0,2.0,0.015910,0.407450,1,0,0,0,3
8,1,1,1,0,1,1,0.0,1.0,1.0,0.0,1.0,0.050244,0.059235,0,0,0,1,3
9,1,1,1,0,1,1,4.0,4.0,4.0,3.0,0.0,0.110728,0.120732,1,0,0,0,0


In [68]:
_ = setup(data, target='RENDIMIENTO_GLOBAL', preprocess=False, train_size=0.9)

,Description,Value
0,Session id,6782
1,Target,RENDIMIENTO_GLOBAL
2,Target type,Multiclass
3,Original data shape,"(692500, 18)"
4,Transformed data shape,"(692500, 18)"
5,Transformed train set shape,"(623250, 18)"
6,Transformed test set shape,"(69250, 18)"
7,Numeric features,17


/Users/andre/miniforge3/envs/hackml/lib/python3.9/site-packages/pycaret/internal/metrics.py:51: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


In [69]:
best_model = compare_models(cross_validation=False)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.4222,0.6936,0.4222,0.4060,0.4092,0.2292,0.2312,33.5900
xgboost,Extreme Gradient Boosting,0.4202,0.6921,0.4202,0.4044,0.4079,0.2266,0.2283,2.9900
lightgbm,Light Gradient Boosting Machine,0.4178,0.6879,0.4178,0.4008,0.4034,0.2233,0.2256,3.8400
gbc,Gradient Boosting Classifier,0.4031,0.6709,0.4031,0.3865,0.3879,0.2037,0.2063,113.2400
ada,Ada Boost Classifier,0.3857,0.6471,0.3857,0.3682,0.3694,0.1805,0.1831,7.4400
lr,Logistic Regression,0.3582,0.6211,0.3582,0.3420,0.3388,0.1435,0.1467,5.9500
rf,Random Forest Classifier,0.3574,0.6189,0.3574,0.3535,0.3550,0.1430,0.1431,7.5200
lda,Linear Discriminant Analysis,0.3561,0.6193,0.3561,0.3423,0.3420,0.1408,0.1428,0.2100
ridge,Ridge Classifier,0.3552,0.0000,0.3552,0.3368,0.3237,0.1391,0.1453,0.1000
dt,Decision Tree Classifier,0.3496,0.5700,0.3496,0.3423,0.3447,0.1325,0.1328,1.4500


/Users/andre/miniforge3/envs/hackml/lib/python3.9/site-packages/pycaret/internal/pycaret_experiment/supervised_experiment.py:323: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  master_display_.apply(


In [55]:
df_test = pd.read_csv('data/test.csv')
df_test.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE
0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Si,No,N,Si,Si,Si,Primaria completa
1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Técnica o tecnológica completa
2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa
3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Si,No,N,No,No,No,Primaria incompleta
4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa


In [37]:
X_test = preprocess_features(df_test)

In [53]:
preds = best_model.predict(X_test).reshape(-1)
final_preds = label_encoder.inverse_transform(preds)

In [56]:
submission_df = pd.DataFrame({
    'ID': df_test.ID,
    'RENDIMIENTO_GLOBAL': final_preds
})

In [61]:
submission_df.to_csv('submissions/submission_03.csv', index=False)

In [67]:
0.2 * df.shape[0]

138500.0